In [92]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from keras.src.layers import Dropout
from keras import Model, Input

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable, GeocoderInsufficientPrivileges
from math import radians, sin, cos, sqrt, atan2

In [16]:
# Load your data
df = pd.read_csv("jobs_in_data.csv")

# Split the 'salary_in_usd' column by 1000
df['salary_in_usd'] /= 1000

# Define features and target variable
X_numerical = df.select_dtypes(include=np.number).drop(
    columns=["salary_in_usd", "salary"])  # Select only numeric columns
y = df["salary_in_usd"]

# Scale numerical data
scaler = StandardScaler()
X_numerical_scaled = scaler.fit_transform(X_numerical)

# Split data into train and test sets
X_train_num, X_test_num, y_train, y_test = train_test_split(X_numerical_scaled, y, test_size=0.2, random_state=42)

# Reshape the input data to include timestep dimension
X_train_reshaped = X_train_num.reshape(X_train_num.shape[0], X_train_num.shape[1], 1)
X_test_reshaped = X_test_num.reshape(X_test_num.shape[0], X_test_num.shape[1], 1)

# Define LSTM model
input_layer = Input(shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]))
lstm_layer = LSTM(128, return_sequences=True)(input_layer)
dropout_layer = Dropout(0.2)(lstm_layer)
lstm_layer2 = LSTM(64)(dropout_layer)
output_layer = Dense(1, activation='linear')(lstm_layer2)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model
y_pred = model.predict(X_test_reshaped)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", np.sqrt(mse))


Epoch 1/100
211/211 [==============================] - 18s 30ms/step - loss: 23861.4863 - val_loss: 20065.5254
Epoch 2/100
211/211 [==============================] - 6s 27ms/step - loss: 18148.7207 - val_loss: 16717.2148
Epoch 3/100
211/211 [==============================] - 4s 19ms/step - loss: 15423.0566 - val_loss: 14384.2949
Epoch 4/100
211/211 [==============================] - 4s 18ms/step - loss: 13312.9268 - val_loss: 12474.0977
Epoch 5/100
211/211 [==============================] - 4s 19ms/step - loss: 11561.8047 - val_loss: 10871.6084
Epoch 6/100
211/211 [==============================] - 3s 13ms/step - loss: 10086.5498 - val_loss: 9516.6992
Epoch 7/100
211/211 [==============================] - 3s 13ms/step - loss: 8845.8984 - val_loss: 8385.4111
Epoch 8/100
211/211 [==============================] - 3s 15ms/step - loss: 7804.8374 - val_loss: 7436.6997
Epoch 9/100
211/211 [==============================] - 3s 13ms/step - loss: 6939.0742 - val_loss: 6651.1621
Epoch 10/100
211

Mean Squared Error: 4084.3665521731436
Root Mean Squared Error: 63.909049063283234

['salary_in_usd'] / ['salary'] 


In [17]:
# Load your data
df = pd.read_csv("jobs_in_data.csv")

# Split the 'salary_in_usd' column by 1000
df['salary_in_usd'] /= 1000

# Calculate the ratio of "salary_in_usd" to "salary"
df['salary_ratio'] = df['salary_in_usd'] / df['salary']

# Define features and target variable
X_numerical = df.select_dtypes(include=np.number).drop(
    columns=["salary_in_usd", "salary"])  # Select only numeric columns
y = df["salary_in_usd"]

# Scale numerical data
scaler = StandardScaler()
X_numerical_scaled = scaler.fit_transform(X_numerical)

# Split data into train and test sets
X_train_num, X_test_num, y_train, y_test = train_test_split(X_numerical_scaled, y, test_size=0.2, random_state=42)

# Reshape the input data to include timestep dimension
X_train_reshaped = X_train_num.reshape(X_train_num.shape[0], X_train_num.shape[1], 1)
X_test_reshaped = X_test_num.reshape(X_test_num.shape[0], X_test_num.shape[1], 1)

# Define LSTM model
input_layer = Input(shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]))
lstm_layer = LSTM(128, return_sequences=True)(input_layer)
dropout_layer = Dropout(0.2)(lstm_layer)
lstm_layer2 = LSTM(64)(dropout_layer)
output_layer = Dense(1, activation='linear')(lstm_layer2)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model
y_pred = model.predict(X_test_reshaped)
mse = mean_squared_error(y_test, y_pred)
# print("MSE:", mse)
print("RMSE:", np.sqrt(mse))


Epoch 1/100
211/211 [==============================] - 19s 30ms/step - loss: 22610.9805 - val_loss: 19545.8242
Epoch 2/100
211/211 [==============================] - 4s 19ms/step - loss: 17630.6074 - val_loss: 16079.5811
Epoch 3/100
211/211 [==============================] - 4s 20ms/step - loss: 14638.8467 - val_loss: 13463.6426
Epoch 4/100
211/211 [==============================] - 4s 20ms/step - loss: 12284.7783 - val_loss: 11352.9814
Epoch 5/100
211/211 [==============================] - 4s 21ms/step - loss: 10375.2920 - val_loss: 9635.1289
Epoch 6/100
211/211 [==============================] - 4s 18ms/step - loss: 8828.1270 - val_loss: 8250.1982
Epoch 7/100
211/211 [==============================] - 4s 18ms/step - loss: 7582.1943 - val_loss: 7140.2505
Epoch 8/100
211/211 [==============================] - 4s 18ms/step - loss: 6595.2783 - val_loss: 6268.6548
Epoch 9/100
211/211 [==============================] - 4s 17ms/step - loss: 5827.7852 - val_loss: 5599.9434
Epoch 10/100
211/2

['salary_in_usd'] / ['salary'] 
RMSE: 60.78381977426994

experience_mapping for ['experience_level']

In [18]:
# Load your data
df = pd.read_csv("jobs_in_data.csv")

# Split the 'salary_in_usd' column by 1000
df['salary_in_usd'] /= 1000

# Calculate the ratio of "salary_in_usd" to "salary"
df['salary_ratio'] = df['salary_in_usd'] / df['salary']

experience_mapping = {
    'Entry-level': 1,
    'Mid-level': 2,
    'Senior': 3,
    'Executive': 4
}

# Map experience levels to ordinal numbers
df['experience_level_encoded'] = df['experience_level'].map(experience_mapping)

# Define features and target variable
X_numerical = df.select_dtypes(include=np.number).drop(
    columns=["salary_in_usd", "salary"])  # Select only numeric columns
y = df["salary_in_usd"]

# Scale numerical data
scaler = StandardScaler()
X_numerical_scaled = scaler.fit_transform(X_numerical)

# Split data into train and test sets
X_train_num, X_test_num, y_train, y_test = train_test_split(X_numerical_scaled, y, test_size=0.2, random_state=42)

# Reshape the input data to include timestep dimension
X_train_reshaped = X_train_num.reshape(X_train_num.shape[0], X_train_num.shape[1], 1)
X_test_reshaped = X_test_num.reshape(X_test_num.shape[0], X_test_num.shape[1], 1)

# Define LSTM model
input_layer = Input(shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]))
lstm_layer = LSTM(128, return_sequences=True)(input_layer)
dropout_layer = Dropout(0.2)(lstm_layer)
lstm_layer2 = LSTM(64)(dropout_layer)
output_layer = Dense(1, activation='linear')(lstm_layer2)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model
y_pred = model.predict(X_test_reshaped)
mse = mean_squared_error(y_test, y_pred)
# print("MSE:", mse)
print("RMSE:", np.sqrt(mse))


Epoch 1/100
211/211 [==============================] - 14s 29ms/step - loss: 22351.6348 - val_loss: 19712.3477
Epoch 2/100
211/211 [==============================] - 4s 20ms/step - loss: 17930.6953 - val_loss: 16432.7637
Epoch 3/100
211/211 [==============================] - 4s 20ms/step - loss: 14972.0225 - val_loss: 13767.4268
Epoch 4/100
211/211 [==============================] - 4s 20ms/step - loss: 12557.3984 - val_loss: 11591.9756
Epoch 5/100
211/211 [==============================] - 4s 20ms/step - loss: 10582.8506 - val_loss: 9813.1934
Epoch 6/100
211/211 [==============================] - 5s 22ms/step - loss: 8975.1074 - val_loss: 8373.9229
Epoch 7/100
211/211 [==============================] - 4s 20ms/step - loss: 7685.1714 - val_loss: 7223.0566
Epoch 8/100
211/211 [==============================] - 4s 21ms/step - loss: 6663.7358 - val_loss: 6325.9531
Epoch 9/100
211/211 [==============================] - 4s 21ms/step - loss: 5870.7251 - val_loss: 5632.1987
Epoch 10/100
211/2

experience_mapping for ['experience_level']
RMSE: 57.54844132269867

distance from  ["employee_residence"] - ["company_location"] 

In [107]:
location_coordinates = {'Germany': (51.1638175, 10.4478313), 'United States': (39.7837304, -100.445882),
                        'United Kingdom': (54.7023545, -3.2765753), 'Canada': (61.0666922, -107.991707),
                        'Spain': (39.3260685, -4.8379791), 'Ireland': (52.865196, -7.9794599),
                        'South Africa': (-28.8166236, 24.991639), 'Poland': (52.215933, 19.134422),
                        'France': (46.603354, 1.8883335), 'Netherlands': (52.2434979, 5.6343227),
                        'Luxembourg': (49.6112768, 6.129799), 'Lithuania': (55.3500003, 23.7499997),
                        'Portugal': (39.6621648, -8.1353519), 'Gibraltar': (36.1285933, -5.3474761),
                        'Australia': (-24.7761086, 134.755), 'Colombia': (4.099917, -72.9088133),
                        'Ukraine': (49.4871968, 31.2718321), 'Slovenia': (46.1199444, 14.8153333),
                        'Romania': (45.9852129, 24.6859225), 'Greece': (43.2097838, -77.6930602),
                        'India': (22.3511148, 78.6677428), 'Latvia': (56.8406494, 24.7537645),
                        'Mauritius': (-20.2759451, 57.5703566), 'Russia': (64.6863136, 97.7453061),
                        'Italy': (42.6384261, 12.674297), 'South Korea': (36.638392, 127.6961188),
                        'Estonia': (58.7523778, 25.3319078), 'Czech Republic': (49.7439047, 15.3381061),
                        'Brazil': (-10.3333333, -53.2), 'Qatar': (25.3336984, 51.2295295),
                        'Kenya': (1.4419683, 38.4313975), 'Denmark': (55.670249, 10.3333283),
                        'Ghana': (8.0300284, -1.0800271), 'Sweden': (59.6749712, 14.5208584),
                        'Turkey': (38.9597594, 34.9249653), 'Switzerland': (46.7985624, 8.2319736),
                        'Andorra': (42.5407167, 1.5732033), 'Ecuador': (-1.3397668, -79.3666965),
                        'Mexico': (19.4326296, -99.1331785), 'Israel': (30.8124247, 34.8594762),
                        'Nigeria': (9.6000359, 7.9999721), 'Saudi Arabia': (25.6242618, 42.3528328),
                        'Argentina': (-34.9964963, -64.9672817), 'Japan': (36.5748441, 139.2394179),
                        'Central African Republic': (7.0323598, 19.9981227), 'Finland': (63.2467777, 25.9209164),
                        'Singapore': (1.357107, 103.8194992), 'Croatia': (45.3658443, 15.6575209),
                        'Armenia': (4.491976149999999, -75.74135085294314),
                        'Bosnia and Herzegovina': (44.3053476, 17.5961467), 'Pakistan': (30.3308401, 71.247499),
                        'Iran': (32.6475314, 54.5643516), 'Bahamas': (24.7736546, -78.0000547),
                        'Austria': (47.59397, 14.12456), 'Puerto Rico': (18.2247706, -66.4858295),
                        'American Samoa': (-14.297124, -170.7131481), 'Thailand': (13.03876215, 101.70017611907599),
                        'Philippines': (12.7503486, 122.7312101), 'Belgium': (50.6402809, 4.6667145),
                        'Egypt': (26.2540493, 29.2675469), 'Indonesia': (-2.4833826, 117.8902853),
                        'United Arab Emirates': (24.0002488, 53.9994829), 'Malaysia': (4.5693754, 102.2656823),
                        'Honduras': (15.2572432, -86.0755145), 'Algeria': (28.0000272, 2.9999825),
                        'Iraq': (33.0955793, 44.1749775), 'China': (35.000074, 104.999927),
                        'New Zealand': (-41.5000831, 172.8344077), 'Moldova': (47.2879608, 28.5670941),
                        'Malta': (35.8885993, 14.4476911), 'Uganda': (1.5333554, 32.2166578),
                        'Tunisia': (36.8002068, 10.1857757), 'Peru': (-6.8699697, -75.0458515),
                        'Uzbekistan': (41.32373, 63.9528098), 'Georgia': (32.3293809, -83.1137366),
                        'Kuwait': (29.3796532, 47.9734174), 'Cyprus': (34.9174159, 32.889902651331866),
                        'Costa Rica': (10.2735633, -84.0739102), 'Chile': (-31.7613365, -71.3187697),
                        'Bolivia': (-17.0568696, -64.9912286), 'Dominican Republic': (19.0974031, -70.3028026),
                        'Vietnam': (15.9266657, 107.9650855), 'Bulgaria': (42.6073975, 25.4856617),
                        'Jersey': (49.2214561, -2.1358386), 'Serbia': (44.024322850000004, 21.07657433209902),
                        'Hong Kong': (22.350627, 114.1849161)}

# Haversine function
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Radius of the Earth in kilometers

    return distance

df = pd.read_csv("jobs_in_data.csv")

# Calculate distances and add them to the DataFrame
df['distance_company_to_residence'] = df.apply(lambda row: haversine(*location_coordinates[row['company_location']], *location_coordinates[row['employee_residence']]), axis=1)

# Load your data

# Split the 'salary_in_usd' column by 1000
df['salary_in_usd'] /= 1000

# Calculate the ratio of "salary_in_usd" to "salary"
df['salary_ratio'] = df['salary_in_usd'] / df['salary']

experience_mapping = {
    'Entry-level': 1,
    'Mid-level': 2,
    'Senior': 3,
    'Executive': 4
}

# Map experience levels to ordinal numbers
df['experience_level_encoded'] = df['experience_level'].map(experience_mapping)


# Calculate the percentile rank of each salary within its job category
df['Percentile'] = df.groupby('job_category')['salary'].rank(pct=True)

# Normalize the percentile ranks to a scale of 0 to 1
min_percentile = df['Percentile'].min()
max_percentile = df['Percentile'].max()
df['Normalized_Salary_within_Job_Category'] = (df['Percentile'] - min_percentile) / (max_percentile - min_percentile)

# Drop the temporary 'Percentile' column if you don't need it anymore
df.drop(columns=['Percentile'], inplace=True)

# Define features and target variable
X_numerical = df.select_dtypes(include=np.number).drop(
    columns=["salary_in_usd", "salary"])  # Select only numeric columns
y = df["salary_in_usd"]

# Scale numerical data
scaler = StandardScaler()
X_numerical_scaled = scaler.fit_transform(X_numerical)

# Split data into train and test sets
X_train_num, X_test_num, y_train, y_test = train_test_split(X_numerical_scaled, y, test_size=0.2, random_state=42)

# Reshape the input data to include timestep dimension
X_train_reshaped = X_train_num.reshape(X_train_num.shape[0], X_train_num.shape[1], 1)
X_test_reshaped = X_test_num.reshape(X_test_num.shape[0], X_test_num.shape[1], 1)

# Define LSTM model
input_layer = Input(shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]))
lstm_layer = LSTM(128, return_sequences=True)(input_layer)
dropout_layer = Dropout(0.2)(lstm_layer)
lstm_layer2 = LSTM(64)(dropout_layer)
output_layer = Dense(1, activation='linear')(lstm_layer2)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model
y_pred = model.predict(X_test_reshaped)
mse = mean_squared_error(y_test, y_pred)
# print("MSE:", mse)
print("RMSE:", np.sqrt(mse))





Epoch 1/100
211/211 [==============================] - 5s 10ms/step - loss: 22209.9219 - val_loss: 19905.7305
Epoch 2/100
211/211 [==============================] - 2s 7ms/step - loss: 18140.7090 - val_loss: 16666.3691
Epoch 3/100
211/211 [==============================] - 2s 8ms/step - loss: 15220.4756 - val_loss: 14026.5635
Epoch 4/100
211/211 [==============================] - 2s 8ms/step - loss: 12806.6582 - val_loss: 11833.3740
Epoch 5/100
211/211 [==============================] - 2s 8ms/step - loss: 10813.9014 - val_loss: 10034.2988
Epoch 6/100
211/211 [==============================] - 2s 8ms/step - loss: 9181.3184 - val_loss: 8564.7402
Epoch 7/100
211/211 [==============================] - 2s 8ms/step - loss: 7859.4482 - val_loss: 7382.5059
Epoch 8/100
211/211 [==============================] - 2s 8ms/step - loss: 6807.5386 - val_loss: 6454.1001
Epoch 9/100
211/211 [==============================] - 2s 9ms/step - loss: 5985.7378 - val_loss: 5735.3140
Epoch 10/100
211/211 [====

Normalized Salary within Job Category
RMSE: 25.531453514062534